<span style='color:red'> This notebook is copied from "hunspell-run-evaluation.ipynb". Hunspell is replaced by Nuspell spell checker. </span>

# Spelling correction with nuspell on litkey data set

Contents:
1. Preparation
    - Imports
    - Load data set
        - **Types**
        - **Token**
    - Preprocessing
2. Running nuspell on litkey data set
    - **Types**
    - **Tokens**
3. Evaluation

## 1 - Preparation

In [1]:
# IMPORTS
#%autoreload 2

import re
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from tqdm.notebook import tqdm # A fast, extensible progress bar

import sys

sys.path.insert(0, '../../')
# sys.path

from Lisa import litkey_2
#import Lisa.litkey_2

In [2]:
# Configuration
# Set figure size for sns plot
%config InlineBackend.figure_format = 'retina'
mpl.rc('figure', figsize=(8, 6), dpi=100)
sns.set()
sns.set_style('darkgrid')

# Set tqdm on pandas
tqdm.pandas(desc="Progress so far...")

# Do not truncate columns of DataFrame
pd.set_option('display.max_rows', None)

In [3]:
#tqdm.pandas(desc='my bar')
#import numpy as np
#tqdm.pandas()
#df = pd.DataFrame(np.random.randint(0, 100, (100000, 6)))
#df.progress_apply(lambda x: x**2)

In [4]:
# LOAD LITKEY DATA SET
# It was the wrong path; Go two levels up
data_error_types = litkey_2.load(litkey_data_path ='../../litkey-data/')

data_error_token = litkey_2.load(litkey_data_path ='../../litkey-data/', toss_duplicates=False) 

In [19]:
# PREPROCESSING

#Hunspell treats words with some punctuation marks as two words (e.g. Seil=bahn), which would destroy the indices; 
#therefore, "=" is replaced with "-" and the others are deleted prior to analysis with Hunspell

data_error_types['unchanged_original'] = data_error_types['original']
#data_error_types['original'] = data_error_types['original'].str.replace("=", '-')
data_error_types['original'] = data_error_types['original'].str.replace("=", '')
data_error_types['original'] = data_error_types['original'].str.replace("-", '') # "-" had to be removed  additionally 
#data_error_types['original'] = data_error_types['original'].str.replace(" ", '')


data_error_types['original'] = data_error_types['original'].str.replace('"', '')
data_error_types['original'] = data_error_types['original'].str.replace(":", '')
data_error_types['original'] = data_error_types['original'].str.replace(",", '')
data_error_types['original'] = data_error_types['original'].str.replace("'", '')

data_error_token['unchanged_original'] = data_error_token['original']
#data_error_token['original'] = data_error_token['original'].str.replace("=", '-')
data_error_token['original'] = data_error_token['original'].str.replace("=", '')
data_error_token['original'] = data_error_token['original'].str.replace("-", '') # "-" had to be removed  additionally 
#data_error_token['original'] = data_error_token['original'].str.replace(" ", '')

data_error_token['original'] = data_error_token['original'].str.replace('"', '')
data_error_token['original'] = data_error_token['original'].str.replace(":", '')
data_error_token['original'] = data_error_token['original'].str.replace(",", '')
data_error_token['original'] = data_error_token['original'].str.replace("'", '')

In [20]:
#display(data_error_types.head(100))
#display(data_error_token.head(100))

In [24]:
data_error_types[~data_error_types.original.str.isalpha()]

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original
2490,weg.gerant,weggerannt,03-346-3-I-Frosch.csv,1,12,1,weg.gerant
5614,1234schulle,1234-Schule,06-477-3-IV-Weg.csv,1,1,1,1234schulle
6415,gesprun.gen,gesprungen,07-339-4-I-Schule.csv,1,26,1,gesprun.gen
6776,U.bahn,U-Bahn,08-021-4-II-Fundbuero.csv,1,5,1,U.bahn
6903,her.,Herr,08-075-4-II-Fundbuero.csv,1,20,1,her.
6941,Herr.,Herr,08-097-4-II-Fundbuero.csv,4,20,4,Herr.
7042,gelb.orange,gelb-orange,08-150-4-II-Fundbuero.csv,1,1,1,gelb.orange
7324,k.no,Kino,08-275-4-II-Fundbuero.csv,1,1,1,k.no
7480,u.,und,08-369-4-II-Fundbuero.csv,1,106,1,u.
7520,min.,min,08-387-4-II-Fundbuero.csv,1,1,1,min.


In [32]:
data_error_types[data_error_types.original.str.isnumeric()]

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original


In [29]:
data_error_types[~data_error_types.original.str.isalnum()]

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original
2490,weg.gerant,weggerannt,03-346-3-I-Frosch.csv,1,12,1,weg.gerant
6415,gesprun.gen,gesprungen,07-339-4-I-Schule.csv,1,26,1,gesprun.gen
6776,U.bahn,U-Bahn,08-021-4-II-Fundbuero.csv,1,5,1,U.bahn
6903,her.,Herr,08-075-4-II-Fundbuero.csv,1,20,1,her.
6941,Herr.,Herr,08-097-4-II-Fundbuero.csv,4,20,4,Herr.
7042,gelb.orange,gelb-orange,08-150-4-II-Fundbuero.csv,1,1,1,gelb.orange
7324,k.no,Kino,08-275-4-II-Fundbuero.csv,1,1,1,k.no
7480,u.,und,08-369-4-II-Fundbuero.csv,1,106,1,u.
7520,min.,min,08-387-4-II-Fundbuero.csv,1,1,1,min.
7653,(Wao,Wau,08-471-4-II-Fundbuero.csv,1,13,1,(Wao


In [30]:
data_error_types[data_error_types.original.str.contains(r' ')]

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original


In [31]:
data_error_types[data_error_types.original.str.isspace()]

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original


## 2 - Running nuspell on litkey data set
###  a) Types

In [ ]:
import sys
print(sys.version)

In [33]:
# Create csv-file containing 'original'-column (misspelled) data to be analysed with nuspell
data_error_types.original.to_csv('original_nu.csv', header=False, index=False)

# The beforementioned csv-file is input to nuspell analysis with german dictionary;
# Output of analysis is put to a txt file
# Build a file 'hunspell.txt' with corrections for each word
!cat original_nu.csv | nuspell -d de_DE > output_nu.txt
#!cat original_nu.csv | nuspell -d de_DE > output_nu.txt



# Build a list (suggestions-column) of top correction suggestions from nuspell for each word, '' for correct, '?' for unrecognized
# Reading in all the lines
hs = []
with open('output_nu.txt') as f:
    #next(f) # Why did this work/was necessary in hunspell version though?
    for index, l in enumerate([l for l in [line.strip() for line in f] if l]):
        # Words recognized as correct
        # (* = dictionary stem (e. g. "man"), + = affixed forms of the following dictionary stem (e. g. "wollt" - wollen))
        # Append nothing to hs
        
        # TODO: Why minus flag?
        if re.match(r'\+|\*|-', l):
            hs.append('')
            
        # Words not recognized/rejected words (# = without suggestions)
        # Append '?' to hs
        elif re.match('#', l):
            hs.append('?')
            
        # Words not recognized/rejected words (& = with suggestions)
        # Append suggested words to hs
        else:
            hs.append(l.split(': ')[1].split(', '))

            
# Add nuspell's corrections as column to data
print(data_error_types.shape[0])
print(len(hs))

data_error_types['suggestions'] = hs

data_error_types.head(40)    # Print excerpt from DataFrame

INFO: Locale LC_CTYPE=de_DE.UTF-8, Used encoding=UTF-8
INFO: Pointed dictionary /Users/lisaprepens/Library/Spelling/de_DE.{dic,aff}
9484
9484


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original,suggestions
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,belt,"[lebt, bellt, bebt, Welt, Zelt, bettelt, Betel]"
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,kukt,"[kurt, bukt, Kult]"
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,..."
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,gekricht,"[gekracht, gekreischt, Gericht, gerichtet, ger..."
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,leken,"[lenken, lecken, ekeln, lenke, lesen, legen, l..."
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,felt,"[fehlt, fielt, feilt, elft, fest, fett, fegt, ..."
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,wolte,"[wollte, walte, wolle, holte]"
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,lekt,"[lenkt, leckt, lest, legt, lebt, Sekt, Lektor]"
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,fom,"[vom, Rom, Tom, Dom, Form]"
9,gawen,kaufen,01-006-2-III-Eis.csv,1,6,1,gawen,"[wagen, garen, gaben, Gazen]"


In [37]:
data_error_types.head(2)
data_error_types.tail(2)

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original,suggestions
9482,mutzte,nutzte,10-693-4-IV-Weg.csv,1,1,1,mutzte,"[mutete, nutzte, motzte, putzte, zumute, Mute,..."
9483,momen,Moment,10-693-4-IV-Weg.csv,1,4,1,momen,"[Nomen, Domen, -omen, Moment, Omen]"


### b) Token

In [39]:
# Create csv-file containing 'original'-column (misspelled) data to be analysed with nuspell
data_error_token.original.to_csv('original_nu_token.csv', header=False, index=False)

# The beforementioned csv-file is input to nuspell analysis with german dictionary;
# Output of analysis is put to a txt file
# Build a file 'hunspell.txt' with corrections for each word
!cat original_nu_token.csv | nuspell -d de_DE > output_nu_token.txt
#!type original_hun.csv | hunspell -d de_DE > hunspell.txt



# Build a list (suggestions-column) of top correction suggestions from nuspell for each word, '' for correct, '?' for unrecognized
# Reading in all the lines
hs = []
with open('output_nu_token.txt') as f:
    #next(f)
    for index, l in enumerate([l for l in [line.strip() for line in f] if l]):
        # Words recognized as correct
        # (* = dictionary stem (e. g. "man"), + = affixed forms of the following dictionary stem (e. g. "wollt" - wollen))
        # Append nothing to hs
        
        # TODO: Why minus flag?
        if re.match(r'\+|\*|-', l):
            hs.append('')
            
        # Words not recognized/rejected words (# = without suggestions)
        # Append '?' to hs
        elif re.match('#', l):
            hs.append('?')
            
        # Words not recognized/rejected words (& = with suggestions)
        # Append suggested words to hs
        else:
            hs.append(l.split(': ')[1].split(', '))

            
# Add nuspell's corrections as column to data
print(data_error_token.shape[0])
print(len(hs))

data_error_token['suggestions'] = hs

data_error_token.head(40)    # Print excerpt from DataFrame

INFO: Locale LC_CTYPE=de_DE.UTF-8, Used encoding=UTF-8
INFO: Pointed dictionary /Users/lisaprepens/Library/Spelling/de_DE.{dic,aff}
24601
24601


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original,suggestions
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,belt,"[lebt, bellt, bebt, Welt, Zelt, bettelt, Betel]"
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,kukt,"[kurt, bukt, Kult]"
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,..."
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,gekricht,"[gekracht, gekreischt, Gericht, gerichtet, ger..."
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,leken,"[lenken, lecken, ekeln, lenke, lesen, legen, l..."
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,felt,"[fehlt, fielt, feilt, elft, fest, fett, fegt, ..."
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,wolte,"[wollte, walte, wolle, holte]"
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,lekt,"[lenkt, leckt, lest, legt, lebt, Sekt, Lektor]"
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,fom,"[vom, Rom, Tom, Dom, Form]"
9,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,..."


In [41]:
# Write pickle
#data_error_token.to_pickle('data_error_token_nuspell_suggestions.pkl')
#data_error_types.to_pickle('data_error_types_nuspell_suggestions.pkl')

## 3 - Evaluation

- **Baseline**:<br>
The upper bound performance is created by checking whether the **target word** (= _**'corrected'**_) is in the **dictionary** (= _**'in_dict'**_) that is used by nuspell itself. It is checked, whether the word can basically be found. <br>
<span style="color:red"> **Since nuspell uses hunspell dictionary, upper bound is the same for both.**_ </span>


- **Furthermore, the following variables are created**:
    - Whether the target word is among the suggested words by hunspell (= _**'in_sugg'**_).
        - At which index it is among the suggested words by hunspell (= _**'sugg_index'**_).
    - Whether the target word is ranked at index 0 by hunspell (= _**'idx0'**_).
    - 'original_cs', 'corrected_cs', 'suggestions_cs': All columns (3) in case sensitive (distinction upper- and lower case)

In [2]:
# Read pickle
data_error_token = pd.read_pickle('data_error_token_nuspell_suggestions.pkl')
data_error_types = pd.read_pickle('data_error_types_nuspell_suggestions.pkl')

In [45]:
#display(data_error_token.head(20))
#display(data_error_token.tail(20))
# no suggestion: empty cell

In [5]:
# Check if empty suggestion cell is nan
#data_error_token.info()
#data_error_token[data_error_token.suggestions.isna()]
# EMPTY CELL CONTAINS EMPTY STRING, SEE ABOVE

In [10]:
data_error_token.suggestions.iloc[0]
data_error_token.corrected.iloc[0]
data_error_token.suggestions.iloc[2].index(data_error_token.corrected.iloc[2]) # (bellt on index 1, dann on index 2) TRUE

#data_ready.corrected.iloc[2]
#.index(data_ready.corrected.iloc[2]) # bellt 1
#(data_ready.corrected.iloc[0])
#data_ready.corrected.iloc[0]

2

In [46]:
data_error_types_lc = litkey_2.evaluate(data_error_types)
data_error_token_lc = litkey_2.evaluate(data_error_token)

data_error_types_cs = litkey_2.evaluate(data_error_types, lower_case=False)
data_error_token_cs = litkey_2.evaluate(data_error_token, lower_case=False)

  0%|          | 0/9484 [00:00<?, ?it/s]

  0%|          | 0/24601 [00:00<?, ?it/s]

  0%|          | 0/9484 [00:00<?, ?it/s]

  0%|          | 0/24601 [00:00<?, ?it/s]

In [47]:
display(data_error_types_lc.head(20))
display(data_error_types_cs.head(20))

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,belt,"[lebt, bellt, bebt, welt, zelt, bettelt, betel]",True,1.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,kukt,"[kurt, bukt, kult]",False,NaN,False
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,...",True,2.0,False
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,gekricht,"[gekracht, gekreischt, gericht, gerichtet, ger...",False,NaN,False
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,leken,"[lenken, lecken, ekeln, lenke, lesen, legen, l...",True,1.0,False
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,felt,"[fehlt, fielt, feilt, elft, fest, fett, fegt, ...",False,NaN,False
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,wolte,"[wollte, walte, wolle, holte]",True,0.0,True
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,lekt,"[lenkt, leckt, lest, legt, lebt, sekt, lektor]",True,1.0,False
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,fom,"[vom, rom, tom, dom, form]",True,0.0,True
9,gawen,kaufen,01-006-2-III-Eis.csv,1,6,1,gawen,"[wagen, garen, gaben, gazen]",False,NaN,False


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,belt,"[lebt, bellt, bebt, Welt, Zelt, bettelt, Betel]",True,1.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,kukt,"[kurt, bukt, Kult]",False,NaN,False
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,...",True,2.0,False
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,gekricht,"[gekracht, gekreischt, Gericht, gerichtet, ger...",False,NaN,False
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,leken,"[lenken, lecken, ekeln, lenke, lesen, legen, l...",True,1.0,False
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,felt,"[fehlt, fielt, feilt, elft, fest, fett, fegt, ...",False,NaN,False
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,wolte,"[wollte, walte, wolle, holte]",True,0.0,True
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,lekt,"[lenkt, leckt, lest, legt, lebt, Sekt, Lektor]",True,1.0,False
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,fom,"[vom, Rom, Tom, Dom, Form]",True,0.0,True
9,gawen,kaufen,01-006-2-III-Eis.csv,1,6,1,gawen,"[wagen, garen, gaben, Gazen]",False,NaN,False


In [48]:
display(data_error_token_lc.head(20))
display(data_error_token_cs.head(20))

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,belt,"[lebt, bellt, bebt, welt, zelt, bettelt, betel]",True,1.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,kukt,"[kurt, bukt, kult]",False,NaN,False
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,...",True,2.0,False
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,gekricht,"[gekracht, gekreischt, gericht, gerichtet, ger...",False,NaN,False
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,leken,"[lenken, lecken, ekeln, lenke, lesen, legen, l...",True,1.0,False
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,felt,"[fehlt, fielt, feilt, elft, fest, fett, fegt, ...",False,NaN,False
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,wolte,"[wollte, walte, wolle, holte]",True,0.0,True
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,lekt,"[lenkt, leckt, lest, legt, lebt, sekt, lektor]",True,1.0,False
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,fom,"[vom, rom, tom, dom, form]",True,0.0,True
9,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,...",True,2.0,False


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_original,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,belt,"[lebt, bellt, bebt, Welt, Zelt, bettelt, Betel]",True,1.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,kukt,"[kurt, bukt, Kult]",False,NaN,False
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,...",True,2.0,False
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,gekricht,"[gekracht, gekreischt, Gericht, gerichtet, ger...",False,NaN,False
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,leken,"[lenken, lecken, ekeln, lenke, lesen, legen, l...",True,1.0,False
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,felt,"[fehlt, fielt, feilt, elft, fest, fett, fegt, ...",False,NaN,False
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,wolte,"[wollte, walte, wolle, holte]",True,0.0,True
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,lekt,"[lenkt, leckt, lest, legt, lebt, Sekt, Lektor]",True,1.0,False
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,fom,"[vom, Rom, Tom, Dom, Form]",True,0.0,True
9,dan,dann,01-005-2-III-Eis.csv,627,651,621,dan,"[an, da, dann, dran, dank, den, das, ran, dar,...",True,2.0,False


In [49]:
# Serialize dataframe, so that analysis does not have to be run every single time
# Dataframe can then be deserialized in another notebook
data_error_token_lc.to_pickle('data_error_token_nuspell_evaluation_lc.pkl')
data_error_types_lc.to_pickle('data_error_types_nuspell_evaluation_lc.pkl')

data_error_token_cs.to_pickle('data_error_token_nuspell_evaluation_cs.pkl')
data_error_types_cs.to_pickle('data_error_types_nuspell_evaluation_cs.pkl')

In [ ]:
# Reload pickle
# Load df from pickle

#data_error_token = pd.read_pickle('data_error_token_nuspell_evaluation.pkl')
#data_error_types = pd.read_pickle('data_error_types_nuspell_evaluation.pkl')